<a href="https://colab.research.google.com/github/Shreyasii11/Fruit-Counting-with-ML-using-HOG-and-SIFT/blob/model-2/HOG_with_Kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Perform KMeans clustering
kmeans = KMeans(n_clusters=2, random_state=0).fit(fd.reshape(-1,1))
orange_cluster = np.argmax(kmeans.cluster_centers_)



# Calculate accuracy and precision using cross-validation
X = fd.reshape(-1,1)
y = np.zeros(fd.shape[0], dtype=np.uint8)
y[kmeans.predict(X) == orange_cluster] = 1
scores_acc = cross_val_score(KMeans(n_clusters=2, random_state=0), X, y, cv=5, scoring='accuracy')
scores_prec = cross_val_score(KMeans(n_clusters=2, random_state=0), X, y, cv=5, scoring='precision')

# Print results
print("Orange count:", np.count_nonzero(y == 1))
print("Accuracy:", scores_acc.mean())
print("Precision:", scores_prec.mean())


In [ ]:
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score
from sklearn.model_selection import cross_val_score

In [ ]:

# Load image and convert to grayscale
img = cv2.imread('/content/Boost-Orange-Yield3.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
# Perform HOG feature extraction
fd, hog_image = hog(gray, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True, multichannel=False)


<ipython-input-9-fcc25ad76994>:2: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd, hog_image = hog(gray, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True, multichannel=False)


In [ ]:
# Reshape feature array
n_features = fd.shape[2]
fd_reshape = fd.reshape(-1, n_features)



IndexError: ignored

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from skimage.feature import hog

# Load image
img = cv2.imread('/content/Valencia_Orange_Tree-50_600x600.jpg')

# Convert image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Extract HOG features
fd, hog_image = hog(gray, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=False)

# Apply K-means clustering
n_samples, n_features = fd.shape
fd_2d = fd.reshape(n_samples, -1)
kmeans = KMeans(n_clusters=2, random_state=0).fit(fd_2d)

# Determine orange cluster label
orange_cluster = 1 if np.sum(kmeans.labels_ == 0) > np.sum(kmeans.labels_ == 1) else 0

# Apply mask to original image
mask = np.zeros(gray.shape[:2], dtype=np.uint8)
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        if kmeans.predict(fd[i,j].reshape(1,-1)) == orange_cluster:
            mask[i,j] = 255
res = cv2.bitwise_and(img,img,mask = mask)

# Count oranges
contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
orange_count = len(contours)

# Calculate accuracy
true_orange_count = 7  # Replace with actual ground truth count
accuracy = orange_count / true_orange_count

# Print results
print(f"Number of oranges: {orange_count}")
print(f"Accuracy: {accuracy}")


<ipython-input-14-807d2fe3b9c3>:13: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd, hog_image = hog(gray, orientations=8, pixels_per_cell=(16, 16),


ValueError: ignored

## **NEXT TRY**

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from skimage.feature import hog
from sklearn.metrics import accuracy_score, precision_score, recall_score


In [ ]:
# Load image
img = cv2.imread('/content/oranges-fruit-orange-tree-spain.jpg')

# Convert image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
# Compute HOG features
hog_feats, hog_img = hog(gray, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True)

In [ ]:
# Apply K-means clustering
n_samples, n_features = hog_feats.shape
hog_feats_2d = hog_feats.reshape(n_samples, -1)
kmeans = KMeans(n_clusters=2, random_state=0).fit(hog_feats_2d)

ValueError: ignored

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import os

# Set up parameters for HOG descriptor
WINDOW_SIZE = (64, 64)
HOG_PARAMS = {'_winSize': WINDOW_SIZE,
              '_blockSize': (16, 16),
              '_blockStride': (8, 8),
              '_cellSize': (8, 8),
              '_nbins': 9}

# Load the image dataset
data = []
labels = []
for folder_name in ['Oranges', 'Non-Oranges']:
    folder_path = f'/content/drive/My Drive/Fruit_Dataset/{folder_name}/'
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename))
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_resized = cv2.resize(img_gray, WINDOW_SIZE)
        hog = cv2.HOGDescriptor(**HOG_PARAMS)
        features = hog.compute(img_resized)
        data.append(features)
        labels.append(1 if folder_name == 'Oranges' else 0)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Apply k-means clustering
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X_train)
train_labels = kmeans.labels_

# Train a k-nearest neighbors classifier on the clustered data
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, train_labels)

# Predict labels for test data
test_labels = kmeans.predict(X_test)
y_pred = knn.predict(X_test)

# Calculate accuracy
accuracy = sum(y_pred == y_test) / len(y_test) * 100
print(f"Accuracy: {accuracy:.2f}%")

# Count the number of oranges
orange_count = sum(test_labels)
print(f"Number of oranges: {orange_count}")


Accuracy: 50.00%
Number of oranges: 4


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score


# Parameters for HOG feature extraction
HOG_PARAMS = {'_winSize': (64, 64),
              '_blockSize': (16, 16),
              '_blockStride': (8, 8),
              '_cellSize': (8, 8),
              '_nbins': 9,
              '_derivAperture': 1,
              '_winSigma': -1,
              '_histogramNormType': 0,
              '_L2HysThreshold': 0.2,
              '_gammaCorrection': True,
              '_nlevels': 64,
              '_signedGradient': False}

In [ ]:

# Load the image dataset
data = []
labels = []
for folder_name in ['Oranges', 'Non-Oranges']:
    folder_path = f'/content/drive/My Drive/Fruit_Dataset/{folder_name}/'
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename))
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_resized = cv2.resize(img_gray, WINDOW_SIZE)
        hog = cv2.HOGDescriptor(**HOG_PARAMS)
        features = hog.compute(img_resized)
        data.append(features)
        labels.append(1 if folder_name == 'Oranges' else 0)

In [ ]:
# Train K-means clustering model
kmeans = KMeans(n_clusters=2, random_state=0).fit(data)

# Load test data
test_img = cv2.imread('/test_image.jpg')
test_img_gray = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
test_img_resized = cv2.resize(test_img_gray, HOG_PARAMS['_winSize'])
hog = cv2.HOGDescriptor(**HOG_PARAMS)
test_features = hog.compute(test_img_resized).reshape(1,-1)
test_features = test_features.astype(np.float32)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:


# Predict using trained model
test_pred = kmeans.predict(test_features)

# Count oranges
orange_count = np.sum(test_pred == 1)

# Calculate accuracy on training data
train_pred = kmeans.predict(data)
train_acc = accuracy_score(labels, train_pred)



ValueError: ignored

In [ ]:
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.cluster import KMeans

# Load image and convert to grayscale
img = cv2.imread('/content/orange10.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Extract HOG features
hog_features = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                   cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys')

# Perform KMeans clustering
kmeans = KMeans(n_clusters=2, random_state=0).fit(hog_features.reshape(-1, 1))
labels = kmeans.labels_

# Count oranges
count_oranges = np.count_nonzero(labels == 0)

# Display count and accuracy
ground_truth_count = 10 # Replace with actual ground truth count
accuracy = count_oranges / ground_truth_count * 100
print("Orange count:", count_oranges)
print("Accuracy:", accuracy, "%")


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Orange count: 252021
Accuracy: 2520210.0 %


In [ ]:
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.cluster import KMeans

# Load image and convert to grayscale
img = cv2.imread('orange10.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Preprocess image
blur = cv2.GaussianBlur(gray, (5, 5), 0)
_, thresh = cv2.threshold(blur, 100, 255, cv2.THRESH_BINARY)

# Extract HOG features
hog_features = hog(thresh, orientations=9, pixels_per_cell=(8, 8),
                   cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys')

# Perform KMeans clustering
kmeans = KMeans(n_clusters=2, random_state=0).fit(hog_features.reshape(-1, 1))
labels = kmeans.labels_

# Count oranges
count_oranges = np.count_nonzero(labels == 0)

# Display count and accuracy
ground_truth_count = 10 # Replace with actual ground truth count
accuracy = count_oranges / ground_truth_count * 100
print("Orange count:", count_oranges)
print("Accuracy:", accuracy, "%")


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Orange count: 335995
Accuracy: 3359950.0 %


In [ ]:





# Cluster HOG features using KMeans
n_clusters = 3 # Number of clusters (assumes 3 classes of objects)
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(train_features)

# Load test image and extract HOG features
test_img = cv2.imread('test_image.jpg')
gray = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
hog_features = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                   cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys')

# Predict cluster labels for test image using KMeans
test_label = kmeans.predict([hog_features])[0]

# Count oranges
cluster_labels = kmeans.labels_
orange_cluster = np.argmax(np.bincount(cluster_labels)) # Cluster with most oranges
count_oranges = np.count_nonzero(cluster_labels == orange_cluster)

# Display count and accuracy
ground_truth_count = 10 # Replace with actual ground truth count
accuracy = accuracy_score([test_label], [orange_cluster]) * 100
print("Orange count:", count_oranges)
print("Accuracy:", accuracy, "%")


error: ignored

In [ ]:
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

In [ ]:
# Load training images and extract HOG features
train_imgs = ['/content/orange2.jpg', '/content/orange3.jpg', '/content/orange4.jpg', '/content/orange5.jpg', '/content/orange6.jpg', '/content/orange7.jpg', '/content/orange8.jpg', '/content/orange9.jpg', '/content/orange10.jpg']
train_features = []

In [ ]:
print(train_imgs)

['/content/orange2.jpg', '/content/orange3.jpg', '/content/orange4.jpg', '/content/orange5.jpg', '/content/orange6.jpg', '/content/orange7.jpg', '/content/orange8.jpg', '/content/orange9.jpg', '/content/orange10.jpg']


In [ ]:
for img_path in train_imgs:
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hog_features = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                       cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys')
    train_features.append(hog_features)

error: ignored

# **LAST TRY**

In [ ]:






# Train k-means clustering on the HOG features
kmeans = KMeans(n_clusters=2)
kmeans.fit(train_data)

# Load testing data
test_data = []
test_labels = []

for folder_name in ['test']:
    for i in range(1, 6):
        img = cv2.imread(f'{folder_name}/{i}.jpg')
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_resized = cv2.resize(img_gray, WINDOW_SIZE)
        hog = cv2.HOGDescriptor(**HOG_PARAMS)
        features = hog.compute(img_resized)
        test_data.append(features)
        test_labels.append(1 if folder_name == 'oranges' else 0)

# Predict using k-means clustering
test_pred = kmeans.predict(test_data)

# Count oranges
orange_count = np.sum(test_pred == 1)

# Train KNN classifier on k-means labels
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(kmeans.labels_.reshape(-1, 1), train_labels)

# Test KNN classifier on k-means predictions
test_pred_labels = knn.predict(test_pred.reshape(-1, 1))

# Calculate accuracy
test_acc = np.mean(test_pred_labels == test_labels)

# Display results
print(f'Orange count: {orange_count}')
print(f'Test accuracy: {test_acc * 100:.2f}%')


In [ ]:
import cv2
import os
import numpy as np
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier

# Set up HOG parameters
WINDOW_SIZE = (64, 64)
HOG_PARAMS = {'_winSize': (64, 64),
              '_blockSize': (16, 16),
              '_blockStride': (8, 8),
              '_cellSize': (8, 8),
              '_nbins': 9,
              '_derivAperture': 1,
              '_winSigma': -1,
              '_histogramNormType': 0,
              '_L2HysThreshold': 0.2,
              '_gammaCorrection': True,
              '_nlevels': 64,
              '_signedGradient': False}




In [ ]:
# Load the image dataset
data = []
labels = []
for folder_name in ['Oranges', 'Non-Oranges']:
    folder_path = f'/content/drive/My Drive/Fruit_Dataset/{folder_name}/'
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename))
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_resized = cv2.resize(img_gray, WINDOW_SIZE)
        hog = cv2.HOGDescriptor(**HOG_PARAMS)
        features = hog.compute(img_resized)
        data.append(features)
        labels.append(1 if folder_name == 'Oranges' else 0)

In [ ]:
# Train k-means clustering on the HOG features
kmeans = KMeans(n_clusters=5)
kmeans.fit(data)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=5)

In [ ]:
# Load image
img = cv2.imread('orange10.jpg')
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_resized = cv2.resize(img_gray, WINDOW_SIZE)

# Compute HOG features
hog = cv2.HOGDescriptor(**HOG_PARAMS)
features = hog.compute(img_resized)
features = np.squeeze(features)
features = features.reshape(1, -1)
# Convert input data to float
features = features.astype(float)

# Predict using trained model
test_pred = kmeans.predict(features)

# Count oranges
orange_count = np.sum(test_pred == 1)

# Display results
print(f'Orange count: {orange_count}')

Orange count: 0
